<a href="https://colab.research.google.com/github/morim3/qiskit-optimization/blob/main/my_experiment/compare_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf qiskit-optimization/
!git clone https://github.com/morim3/qiskit-optimization

Cloning into 'qiskit-optimization'...
remote: Enumerating objects: 4199, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 4199 (delta 168), reused 167 (delta 88), pack-reused 3935
Receiving objects: 100% (4199/4199), 5.30 MiB | 22.25 MiB/s, done.
Resolving deltas: 100% (2781/2781), done.


In [2]:
!nvidia-smi
# !pip install -r qiskit-optimization/requirements-dev.txt
!pip install qiskit-aer-gpu
!pip install docplex
!pip install pylatexenc

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("./qiskit-optimization/")

Thu Dec  1 20:38:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    57W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
%load_ext autoreload
%autoreload

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import PQCGroverOptimizer, PBILGroverOptimizer, GroverOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.pqc_grover_optimizer import ProposeDistribution
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import TwoLocal
from docplex.mp.model import Model

import tqdm
import numpy as np
print(Aer.backends())
backend = Aer.get_backend('aer_simulator')
backend.set_options(device='GPU')
backend_statevector = Aer.get_backend('statevector_simulator')
print(backend)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[AerSimulator('aer_simulator'), AerSimulator('aer_simulator_statevector'), AerSimulator('aer_simulator_statevector_gpu'), AerSimulator('aer_simulator_density_matrix'), AerSimulator('aer_simulator_density_matrix_gpu'), AerSimulator('aer_simulator_stabilizer'), AerSimulator('aer_simulator_matrix_product_state'), AerSimulator('aer_simulator_extended_stabilizer'), AerSimulator('aer_simulator_unitary'), AerSimulator('aer_simulator_unitary_gpu'), AerSimulator('aer_simulator_superop'), QasmSimulator('qasm_simulator'), StatevectorSimulator('statevector_simulator'), UnitarySimulator('unitary_simulator'), PulseSimulator('pulse_simulator')]
aer_simulator_gpu


In [23]:
def get_problem(dim):
  import itertools
  model = Model()
  xs = [model.binary_var(name=f"x{i}") for i in range(dim)]
  objective = sum([np.random.randint(1, 4) * x for x in xs]) + \
      sum([np.random.randint(-4, -1) * x * y 
      for x, y in itertools.combinations(xs, 2) 
      if np.random.choice([0, 1], p=[2/3, 1/3])==1])
  # objective = (-xs[0]+2*xs[1]-3*xs[2]-2*xs[0]*xs[2]-1*xs[1]*xs[2])
  model.minimize(objective)
  qp = from_docplex_mp(model)
  return qp

def calc_eval_num(operation_dict, optimal_iteration):
    eval_num = 0
    for i, x in enumerate(operation_dict.values()):
      if i <= optimal_iteration:
        eval_num += x['Q(x)']
        if 'Q' in x:
            eval_num += x['Q']
    return eval_num

def exact_solution(qp):
  exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
  exact_result = exact_solver.solve(qp)
  return exact_result.fval

def pqc_optimizer(qp, exact_fval):
  num_key_qbits = len(qp.objective.linear.to_array())
  init_state = QuantumCircuit(num_key_qbits)
  init_state.h(range(num_key_qbits))
  ansatz = TwoLocal(num_key_qbits, 'ry', 'crx', 'circular', reps=1,
                    insert_barriers=True,
                    initial_state=init_state,
                    skip_final_rotation_layer=True,
                    )
  
  init_param = np.zeros(ansatz.num_parameters)
  propose_distribution = ProposeDistribution(
      ansatz,                 
      num_key_qbits, 
      QuantumInstance(backend=backend, shots=1024, seed_simulator=99),
      init_param=init_param
      )
  grover_optimizer = PQCGroverOptimizer(
      num_value_qubits=8, 
      propose_dist=propose_distribution,
      num_iterations=15, 
      quantum_instance=backend,      
      sample_num=1, quantile=0.,
      threshold_lr=1. ,
      positive_train_conf={"iter": 5, "learning_rate": 0.001},
      negative_train_conf={"iter": 5, "learning_rate": 0.0001})
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = calc_eval_num(results.operation_counts, results._optimal_iteration)
  return is_optimal, eval_num

def grover_optimizer(qp, exact_fval):
  grover_optimizer = GroverOptimizer(8, num_iterations=15, quantum_instance=backend,)
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = calc_eval_num(results.operation_counts, results._optimal_iteration)
  
  return is_optimal, eval_num

def pbil_optimizer(qp, exact_fval):
  grover_optimizer = PBILGroverOptimizer(8, num_iterations=15, quantum_instance=backend, 
                                       pbil_sample_num=1, quantile=0., lr=0.05, )
  results = grover_optimizer.solve(qp)
  is_optimal = results.intermediate_fval <= exact_fval
  eval_num = calc_eval_num(results.operation_counts, results._optimal_iteration)
  return is_optimal, eval_num
  

def compare_performance(dim, exp_num):
  pqc_evals = []
  pbil_evals = []
  grover_evals = []
  for n_exp in tqdm.notebook.tqdm(range(exp_num)):
    
    exact_fval = 1
    while exact_fval >= 0:
      qp = get_problem(dim)
      exact_fval = exact_solution(qp)

    print(qp)
    print("exact fval: ", exact_fval)

    is_optimal, eval_num = pqc_optimizer(qp, exact_fval)
    if is_optimal:
      pqc_evals.append(eval_num)

    is_optimal, eval_num = pbil_optimizer(qp, exact_fval)
    if is_optimal:
      pbil_evals.append(eval_num)

    is_optimal, eval_num = grover_optimizer(qp, exact_fval)
    if is_optimal:
      grover_evals.append(eval_num)

  return np.array(pqc_evals), np.array(pbil_evals), np.array(grover_evals)



In [ ]:
results = []
for dim in [7, 10, 13, 16]:
  results.append(compare_performance(dim, 3))

  0%|          | 0/3 [00:00<?, ?it/s]

minimize -4*x0*x2 - 4*x0*x5 - 3*x2*x3 - 4*x2*x5 - 2*x3*x5 - 3*x5*x6 + x0 + 2*x1 + x2 + 3*x3 + 2*x4 + x5 + x6 (7 variables, 0 constraints, 'docplex_model14')
exact fval:  -13.0
sampled [1 1 0 1 0 0 0]
value 6
real_val 6.0
param:  [ 0.00090622  0.00148158 -0.00219492  0.0004036   0.00219492 -0.00090622
  0.00054159  0.00125493  0.00148158 -0.00219492  0.00054159  0.00054159
  0.00019288 -0.00183029]
sampled [0 0 0 0 1 0 0]
value 2
real_val 2.0
param:  [ 0.0018225   0.00104308 -0.00405119  0.00016917  0.00091401 -0.00134472
  0.00020067  0.00186291  0.00144896 -0.00253584 -0.00016397 -0.00073932
  0.00063138 -0.00081643]
sampled [1 1 1 0 0 0 1]
value 1
real_val 1.0
param:  [ 0.00144949  0.00272929 -0.00352024  0.00185537  0.00061037 -0.00461153
  0.00090564  0.00199252  0.00471576 -0.00266546 -0.00383211 -0.00151367
 -0.00195878 -0.00011146]
sampled [0 0 0 0 1 0 0]
value 2
real_val 2.0
param:  [ 0.00153423  0.00285527 -0.00485327  0.00136476  0.0019434  -0.00506091
 -0.00011909  0.0009677

  0%|          | 0/3 [00:00<?, ?it/s]

minimize -4*x0*x4 - 4*x0*x5 - 4*x1*x2 - 3*x1*x4 - 4*x2*x3 - 4*x2*x4 - 2*x2*x6 - 3*x4*x5 - 3*x4*x7 - 3*x4*x8 - 2*x5*x7 - 3*x6*x8 + x0 + 3*x1 + 2*x2 + x3 + 3*x4 + x5 + 2*x6 + 3*x7 + x8 + x9 (10 variables, 0 constraints, 'docplex_model17')
exact fval:  -22.0
sampled [1 1 1 0 1 0 1 0 1 0]
value -11
real_val -11.0
param:  [ 0.00693096 -0.00693096  0.00693096 -0.00693096 -0.00693096 -0.00693096
 -0.00693096 -0.00693096  0.00693096  0.00693096 -0.00693096 -0.00693096
 -0.00693096  0.00693096 -0.00693096  0.00693096  0.00693096  0.00693096
  0.00693096 -0.00693096]
threshold -11
sampled [0 1 0 0 1 1 1 1 1 0]
value -4
real_val -4.0
param:  [ 0.00733638 -0.00872263  0.00872263 -0.02709608 -0.02570983  0.01323416
 -0.00513929 -0.00652554  0.00513929  0.00872263 -0.00872263  0.01184792
 -0.00733638  0.02709608  0.01323416  0.02570983  0.00652554 -0.01184792
  0.00872263 -0.00733638]
sampled [1 0 1 1 1 1 0 0 1 1]
value -12
real_val -12.0
param:  [-0.0983928   0.08314464  0.29818633 -0.31655978  0.2

In [ ]:
import pickle
with open('drive/MyDrive/Colab Notebooks/1202_compare_methods.pickle','w'):
  pickle.dump(results,f)